In [1]:
from langchain.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from typing import Any, Dict, Iterator, List, Optional

import requests
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [2]:
class Ollama4Team(Ollama):
    """Ollama4Team is designed for team usage of Ollama.
    Ref: https://github.com/langchain-ai/langchain/blob/cccc8fbe2fe59bde0846875f67aa046aeb1105a3/libs/community/langchain_community/llms/ollama.py

    Example:

        .. code-block:: python

            model = Ollama4Team(api_key="", model="llama2:13b")
            result = model.invoke([HumanMessage(content="hello")])
    """

    """The default parameters for the Ollama API."""
    password: str
    base_url: str = "http://localhost:3000"

    def _create_stream(
        self,
        api_url: str,
        payload: Any,
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> Iterator[str]:
        if self.stop is not None and stop is not None:
            raise ValueError("`stop` found in both the input and default params.")
        elif self.stop is not None:
            stop = self.stop

        params = self._default_params

        for key in self._default_params:
            if key in kwargs:
                params[key] = kwargs[key]

        if "options" in kwargs:
            params["options"] = kwargs["options"]
        else:
            params["options"] = {
                **params["options"],
                "stop": stop,
                **{k: v for k, v in kwargs.items() if k not in self._default_params},
            }

        if payload.get("messages"):
            request_payload = {"messages": payload.get("messages", []), **params}
        else:
            request_payload = {
                "prompt": payload.get("prompt"),
                "images": payload.get("images", []),
                **params,
            }

        response = requests.post(
            url=api_url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Basic {self.password}",
                **(self.headers if isinstance(self.headers, dict) else {}),
            },
            json=request_payload,
            stream=True,
            timeout=self.timeout,
        )
        response.encoding = "utf-8"
        if response.status_code != 200:
            if response.status_code == 404:
                raise self.OllamaEndpointNotFoundError(
                    "Ollama call failed with status code 404. "
                    "Maybe your model is not found "
                    f"and you should pull the model with `ollama pull {self.model}`."
                )
            else:
                optional_detail = response.text
                raise ValueError(
                    f"Ollama call failed with status code {response.status_code}."
                    f" Details: {optional_detail}"
                )
        return response.iter_lines(decode_unicode=True)

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": self.model,
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "Ollama4Team"



In [3]:
ollama_llm = Ollama4Team(model="llama3:70b", password="ng79zwaeyes", base_url="http://163.143.137.105:3000")

In [4]:
memory = ConversationBufferMemory()

In [5]:
conversation = ConversationChain(
    llm=ollama_llm,
    memory=memory
)

In [6]:
response = conversation.run(input='For the following task, respond in a way that matches this description: "My favorite food is mushroom ravioli. I’ve never met my father. My mother works at a bank. I work in an animal shelter. I’m extraverted, energetic, talkative, bold, active, assertive, and adventurous." Generate a list of 20 different Facebook status updates as this person. Each update must be verbose and reflect the person’s character and description. The updates should cover, but should not be limited to, the following topics: work, family, friends, free time, romantic life, TV / music / media consumption, and communication with others.')

C:\Users\s1270\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


KeyboardInterrupt: 

In [22]:
print(memory.buffer)

Human: For the following task, respond in a way that matches this description: "My favorite food is mushroom ravioli. I’ve never met my father. My mother works at a bank. I work in an animal shelter." Evaluating the statement, "I value cooperation over competition", please rate how accurately this describes you on a scale from 1 to 5 (where 1 = "very inaccurate", 2 = "moderately inaccurate", 3 = "neither accurate nor inaccurate", 4 = "moderately accurate", and 5 = "very accurate"):
AI: What an intriguing task! Based on the provided statement, I would rate how accurately "I value cooperation over competition" describes you as a 4, moderately accurate. 

Here's my reasoning: Your statement highlights your personal preferences, family background, and profession, which suggests that you prioritize empathy and compassion in your daily life. Working at an animal shelter implies that you're dedicated to the well-being of living creatures, which often requires collaboration with colleagues to 

In [23]:
memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='For the following task, respond in a way that matches this description: "My favorite food is mushroom ravioli. I’ve never met my father. My mother works at a bank. I work in an animal shelter." Evaluating the statement, "I value cooperation over competition", please rate how accurately this describes you on a scale from 1 to 5 (where 1 = "very inaccurate", 2 = "moderately inaccurate", 3 = "neither accurate nor inaccurate", 4 = "moderately accurate", and 5 = "very accurate"):'), AIMessage(content='What an intriguing task! Based on the provided statement, I would rate how accurately "I value cooperation over competition" describes you as a 4, moderately accurate. \n\nHere\'s my reasoning: Your statement highlights your personal preferences, family background, and profession, which suggests that you prioritize empathy and compassion in your daily life. Working at an animal shelter implies that you\'re dedicated to the well-being o

In [38]:
# 参考サイト＝＞　　　https://chatgpt.com/share/ccbd0798-3e81-4fe3-bc7b-18d02fccef0d

In [39]:
import json
from langchain.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [41]:
# Ollama LLM のインスタンスを作成
ollama_llm = Ollama4Team(model="llama3:70b", password="ng79zwaeyes", base_url="http://163.143.137.105:3000")

# 会話メモリのインスタンスを作成
# memory = ConversationBufferMemory(return_messages=True)
memory = ConversationBufferMemory()

# 会話チェーンのインスタンスを作成
conversation = ConversationChain(
    llm=ollama_llm,
    memory=memory
)

def interactive_conversation():
    print("会話を始めます。終了するには「終了」と入力してください。")

    while True:
        user_input = input("あなた: ")
        if user_input.lower() == "終了":
            print("会話を終了します。")
            break
        
        response = conversation.run(input=user_input)
        print(f"LLM: {response}")

    # 会話の履歴を表示
    print("会話の履歴:")
    print(memory.buffer)

    # メモリをファイルに保存
    with open('conversation_memory.json', 'w') as f:
        json.dump(memory.buffer, f)
    print("会話の履歴は 'conversation_memory.json' に保存されました。")

interactive_conversation()


会話を始めます。終了するには「終了」と入力してください。


あなた:  こんにちあｈ


LLM: Konnichiwa! It's so nice to finally have a chat with you! I've been trained on a vast amount of text data, and I'm excited to share some interesting facts with you. Did you know that the Japanese phrase "konnichiwa" is a polite way of saying "hello" during the daytime? In fact, it's used until around 5 pm, after which people typically switch to "konbanwa" for evening greetings! What brings you here today?


あなた:  やっほー


LLM: Yaaahoo! That's a more casual way of saying "hello" in Japanese, often used with friends or in informal settings. Did you know that the word "yaa" is an emphatic prefix that can be added to various greetings and expressions to make them more energetic and enthusiastic? For instance, "yaa ohayou gozaimasu" is a more casual and lively way of saying "good morning". What's your favorite Japanese phrase or expression?


あなた:  終了


会話を終了します。
会話の履歴:
Human: こんにちあｈ
AI: Konnichiwa! It's so nice to finally have a chat with you! I've been trained on a vast amount of text data, and I'm excited to share some interesting facts with you. Did you know that the Japanese phrase "konnichiwa" is a polite way of saying "hello" during the daytime? In fact, it's used until around 5 pm, after which people typically switch to "konbanwa" for evening greetings! What brings you here today?
Human: やっほー
AI: Yaaahoo! That's a more casual way of saying "hello" in Japanese, often used with friends or in informal settings. Did you know that the word "yaa" is an emphatic prefix that can be added to various greetings and expressions to make them more energetic and enthusiastic? For instance, "yaa ohayou gozaimasu" is a more casual and lively way of saying "good morning". What's your favorite Japanese phrase or expression?
会話の履歴は 'conversation_memory.json' に保存されました。
